In [1]:
import pandas as pd
import numpy as np
import requests
import time
import gmaps

#import API key
from apikeys import google_key

#configure gmaps
gmaps.configure(api_key=google_key)

#Read weather/location csv from weatherpy
location_weather = pd.read_csv("cityweather_copy.csv")
locaton_weather = location_weather.dropna(inplace=True)
location_weather.head()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,50.5167,115.1333,33.73,36,0,7.05,RU,1618591504
1,57.7900,-152.4072,39.20,87,90,6.91,US,1618591504
2,-53.1500,-70.9167,48.20,61,40,36.82,CL,1618591505
3,-9.8000,-139.0333,80.56,72,88,19.24,PF,1618591505
4,42.0970,-79.2353,37.99,93,90,13.80,US,1618591505


In [2]:
#Create a heat map that displays the humidity for every city from Part I.
#Store lat and lng in locations
locations = location_weather[["Lat","Lng"]]

#Plot Heatmap

fig = gmaps.figure()

#Create heat map layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
#Add heat layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
location_weather = location_weather[(location_weather['Max Temp'] > 70) & (location_weather['Max Temp'] < 80)]
idealLocation = location_weather[(location_weather['Wind Speed'] < 10) & (location_weather['Cloudiness'] == 0)]
idealLocation

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,-21.3585,55.7679,78.80,78,0,4.61,RE,1618591513
218,31.6100,34.7642,71.01,40,0,9.55,IL,1618591577
410,-21.3393,55.4781,78.80,78,0,4.61,RE,1618591642
420,-17.6394,-71.3375,71.60,78,0,9.22,PE,1618591645
468,-28.5708,-70.7581,78.42,41,0,6.80,CL,1618591382
570,35.2182,52.3409,74.25,16,0,7.85,IR,1618591695


In [8]:
#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
#Create hotel column and loop through the new dataframe using google places api

hotel = []
hotelLat = []
hotelLng =[]


#Set params
radius = 4000
type = 'lodging'

params = {
    'radius': radius,
    'type': type,
    'key': google_key
}


for index,row in idealLocation.iterrows():
   
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    #Grab lat and lng and format to string
    lat = str(row['Lat'])
    lng =str(row['Lng'])
    
    #Update params for each new lat and lng
    params['location'] = f"{lat},{lng}"
    
    response = requests.get(base_url, params).json()
    
    try:
        hotel.append(response["results"][0]["name"])
        hotelLat.append(response["results"][0]["geometry"]["location"]["lat"])
        hotelLng.append(response["results"][0]["geometry"]["location"]["lng"])
    except IndexError:
        hotel.append(None)
        hotelLat.append(None)
        hotelLng.append(None)
    

In [9]:
#Add hotel data into dataframe
idealLocation["Hotel"] = hotel
idealLocation["Hotel Lat"] = hotelLat
idealLocation["Hotel Lng"] = hotelLng
idealLocation

/Users/stevenzych/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/stevenzych/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/stevenzych/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is 

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel,Hotel Lat,Hotel Lng
29,-21.3585,55.7679,78.80,78,0,4.61,RE,1618591513,"Chambres d'hôte ""La Trinité""",-21.358434,55.770322
218,31.6100,34.7642,71.01,40,0,9.55,IL,1618591577,מגדלי סמדר,31.611065,34.766227
410,-21.3393,55.4781,78.80,78,0,4.61,RE,1618591642,Lindsey Hôtel,-21.341006,55.472291
420,-17.6394,-71.3375,71.60,78,0,9.22,PE,1618591645,Torreblanca Hotel,-17.654905,-71.347047
468,-28.5708,-70.7581,78.42,41,0,6.80,CL,1618591382,Humacao Bed & Breakfast,-28.573242,-70.759710
570,35.2182,52.3409,74.25,16,0,7.85,IR,1618591695,Peyvand Hotel,35.229847,52.342858


In [13]:
#Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.
coordinates = tuple(zip(hotelLat,hotelLng))
marker = gmaps.marker_layer(coordinates)
fig.add_layer(marker)
fig

Figure(layout=FigureLayout(height='420px'))